In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import * 
from fastai.text import * 
import numpy as np
import io
import pathlib
import asyncio
from fastai_sentencepiece import *
from filetokenizer import *
from languagemodelloader import *

In [3]:
lang="en"
minToks = 10

In [4]:
#pathData       = Path("../../data/nlp")
pathData       = Path("../nlp-data")
path           = pathData / lang
pathDump       = path/"wiki-dump"
pathJson       = path/"wiki-json"

pathTrainValid = path/"wiki-train_valid"
pathTxt        = pathTrainValid/"txt"
pathToks       = pathTrainValid/"toks"
pathcsv        = pathTrainValid/"wiki.csv"

cache_name   = "sp-model"
pathVocab    = pathTrainValid / cache_name

In [5]:
#Discard section with kess than "minTok" tokens
spt_func  = partial(SentencepieceTokenizer.create, pathVocab=pathVocab)
spt_func.__name__ = SentencepieceTokenizer.__name__
spt       = spt_func(lang="en")
pad_idx   = spt.vocab().numericalize([text.transform.PAD])[0]
vocab,max_vocab = spt.vocab(), len(spt.vocab().itos)
trainTokenizer = FileTokenizer(pathToks/"train", spt_func,"en",vocab,minToks=minToks,n_cpus=max(defaults.cpus-1,1))
validTokenizer = FileTokenizer(pathToks/"valid", spt_func,"en",vocab,minToks=minToks,n_cpus=max(defaults.cpus-1,1))

print(trainTokenizer)
print("size og vocabulary:", max_vocab)
print("pad_idx:",pad_idx)

print(spt.vocab().numericalize( ["xxunk" ,"xxbos","xxeos" ,"xxpad" ,"xxmaj" ,"xxup" ,"xxrep" ,"xxwrep", "xxfld"]  ))
#sentence = ["She is tall.", "He is small"]
#tokenizer._process_all_1(sentence)

self.dtype:<class 'numpy.int16'>
self.dtype:<class 'numpy.int16'>
Tokenizer SentencepieceTokenizer in en with the following rules:
 - fix_html
 - replace_rep
 - replace_wrep
 - spec_add_spaces
 - rm_useless_spaces
 - rm_extra_lineshift
 - replace_all_caps
 - deal_caps

size og vocabulary: 32000
pad_idx: 2
[0, 1, 0, 2, 3, 4, 5, 6, 7]


In [6]:
files   = np.asarray( list(pathTxt.glob("*.txt")) )
nrows   = len(files)
split   = 0.2
splitindex, index = int(nrows*split+.5), np.random.permutation(np.arange(nrows)) 

chunksize=0

trainList = TextList( files[:-splitindex], vocab=vocab, pad_idx=pad_idx, 
                      processor=[FileTokenizeProcessor(tokenizer=trainTokenizer, 
                                                       chunksize=chunksize, mark_fields=False)])

validList = TextList( files[-splitindex:], vocab=vocab, pad_idx=pad_idx, 
                      processor=[FileTokenizeProcessor(tokenizer=validTokenizer, 
                                                       chunksize=chunksize, mark_fields=False)])


In [7]:
%time trainIDS = trainTokenizer.getIds()

CPU times: user 1min 52s, sys: 8.81 s, total: 2min 1s
Wall time: 2min 3s


In [8]:
%time validIDS=validTokenizer.getIds()

CPU times: user 35.4 s, sys: 4.08 s, total: 39.4 s
Wall time: 40.8 s


In [57]:
from languagemodelloader import *
nTrainToks, nValidToks = int(1e1),int(1e1)
#nTrainToks, nValidToks = -1,-1
if nTrainToks>0 and nValidToks>0:
    trainIDS_ = trainIDS[0:nTrainToks]
    validIDS_ = validIDS[0:nValidToks]
else:
    trainIDS_ = trainIDS
    validIDS_ = validIDS

db = MyTextLMDataBunch.from_ids( pathTrainValid, vocab, trainIDS_, validIDS_, bptt=70, p_bptt=0.0, bs=1)
#print(f"len(data.train_dl):{len(data.train_dl)} len(data.train_dl.x.items):{len(data.train_dl.x.items)}")

MyTextLMDataBunch def create
LanguageModelLoader.__init__ batches:17 nToks:1130 bptt:70 p_bptt:0.0 shuffle:True backwards:False
LanguageModelLoader.__init__ batches:32 nToks:2187 bptt:70 p_bptt:0.0 shuffle:False backwards:False
LanguageModelLoader.__init__ batches:17 nToks:1130 bptt:70 p_bptt:0.0 shuffle:False backwards:False


In [58]:
128-49
134-128+1+42
71-42+1+50
70-50+1+30

51

In [ ]:
# %load_ext line_profiler
data = MyLanguageModelLoader(dataset=db.train_dl,bptt=70, p_bptt=0.0, bs=2)
print(data)
            
def getAllBatches(data,epochs=1):
    for i in range(epochs):
        for xb,yb in data:
            continue
%time getAllBatches(data)

In [ ]:
            import tracemalloc
            tracemalloc.start()
            snapshot1 = tracemalloc.take_snapshot()

            -----your interesting code
            
            snapshot2 = tracemalloc.take_snapshot()
            top_stats = snapshot2.compare_to(snapshot1, 'lineno')
            print(f"Top 10 of {len(top_stats)}")
            for stat in top_stats[:10]: print(stat)
          

In [ ]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

In [34]:
#create test data sao that we can contro whether LanguageModelLoader returns contigous tokens 
jagged = []
seq_len = 9
bs = 4
iterations=11

countTokens = 0
totalTokens = bs*seq_len*iterations
print(f"totalTokens:{totalTokens}")
rand_interval=20
while countTokens < totalTokens:
    nb = totalTokens-countTokens if   totalTokens-countTokens <rand_interval\
                                 else 1+int(np.random.random() * rand_interval)
    #print(f"nb:{nb}")
    jagged.append(np.arange(countTokens+1,countTokens+1+nb))
    countTokens = jagged[-1][-1]
jagged = np.asarray(jagged)    
print(f"countTokens:{countTokens}")
for j in jagged: print(j)

totalTokens:396
countTokens:396
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[15 16 17 18]
[19 20 21 22]
[23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]
[41 42]
[43 44 45 46]
[47 48 49 50 51 52 53 54 55 56 57 58 59 60 61]
[62 63 64 65 66 67 68 69 70 71 72]
[73 74 75 76 77 78 79]
[80 81 82 83 84 85 86 87 88 89 90 91 92 93 94]
[ 95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111]
[112 113 114 115 116 117 118 119 120]
[121 122 123 124 125 126 127 128 129 130 131 132 133]
[134 135 136 137 138 139 140 141 142 143]
[144 145 146 147 148 149 150 151 152 153 154 155]
[156 157 158 159 160 161 162 163 164 165 166 167 168 169]
[170 171 172 173 174 175 176 177]
[178 179 180 181 182 183 184 185 186 187 188 189 190]
[191 192 193 194 195 196 197 198 199 200 201 202]
[203 204 205 206 207 208]
[209 210 211 212 213 214 215 216 217 218 219 220]
[221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239]
[240 241 242 243 244 245 246 247 248 249 250 251 252]
[253 254

In [63]:
# %load_ext line_profiler
trainIDS_ = jagged
validIDS_ = jagged
db   = MyTextLMDataBunch.from_ids( pathTrainValid, vocab, trainIDS_, validIDS_, bptt=seq_len, p_bptt=0.0, bs=bs)
data = MyLanguageModelLoader(dataset=db.train_dl, bptt=seq_len, p_bptt=0.0, bs=bs, shuffle=False)
#print(data)
            
def getAllBatches(data,epochs=1):
    x=None
    for i in range(epochs):
        for xb,yb in data:
            if x is None: x = xb[:,0:seq_len].data.numpy()
            else:         x = np.concatenate((x, xb[:,0:seq_len].data.numpy()),axis=1)
            continue
    return x            
%time batches = getAllBatches(data)

MyTextLMDataBunch def create
LanguageModelLoader.__init__ iterations:11 rags:35 nToks:396 bptt:9 p_bptt:0.0 shuffle:True backwards:False
LanguageModelLoader.__init__ iterations:11 rags:35 nToks:396 bptt:9 p_bptt:0.0 shuffle:False backwards:False
LanguageModelLoader.__init__ iterations:11 rags:35 nToks:396 bptt:9 p_bptt:0.0 shuffle:False backwards:False
LanguageModelLoader.__init__ iterations:11 rags:35 nToks:396 bptt:9 p_bptt:0.0 shuffle:False backwards:False
LanguageModelLoader.allocate_buffers shuffle:False backwards:False self.ite_len:11
   ei  eo  length
0   0   0      14
1  10  12      17
2  18   4      12
3  27   2      12
len(self):11 Number of iteration:11


self.ite_len:11 Number of iterations:11 countToks:440 self.totalToks:396 countToks < self.totalToks:False
   ei  eo  length
0  10   5      17
1  19   3       6
2  27   6      12
3  34  10      18



CPU times: user 7.76 ms, sys: 2.54 ms, total: 10.3 ms
Wall time: 8.22 ms


In [64]:
#The input was size to an batches that do not wrap aoround. Also the input contains continuously 
#increasing numbers. Therefore the diff below from one to the next column must be 1 for a properly aligned batches
b_diff = batches[:,1:] - batches[:,:-1]

In [66]:
b_diff.flatten().all()

True